# Imports

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import pylab as plt

# Functions

In [ ]:
def clean_data():
    
    global db1
    db1 = db1[(db1.num_infections > 0)].reset_index()
    db1 = db1.drop(columns = ['index'], axis = 1)
    
    db1['date'] = pd.to_datetime(db1.date)
    
   
    db1['autonomous_region'] = db1.autonomous_region.apply(lambda x: 'Comunidad Valenciana' if x == 'Valenciana, Comunidad' else x)
    db1['autonomous_region'] = db1.autonomous_region.apply(lambda x: 'Comunidad de Madrid' if x == 'Madrid, Comunidad de' else x)
    db1['autonomous_region'] = db1.autonomous_region.apply(lambda x: 'Región de de Murcia' if x == 'Murcia, Región de' else x)
    db1['autonomous_region'] = db1.autonomous_region.apply(lambda x: 'Comunidad Foral de Navarra' if x == 'Navarra, Comunidad Foral de' else x)
    db1['autonomous_region'] = db1.autonomous_region.apply(lambda x: 'Principado de Asturias' if x == 'Asturias, Principado de' else x)
    
    
    db1['province'] = db1.province.apply(lambda x: 'Alicante' if x == 'Alicante/Alacant' else x)
    db1['province'] = db1.province.apply(lambda x: 'Castellón' if x == 'Castellón/Castelló' else x)
    db1['province'] = db1.province.apply(lambda x: 'Araba' if x == 'Araba/Álava' else x)
    
    db1['sex'] = db1.sex.apply(lambda x: 'UKNOWN' if x == 'NC' else x)
    
    db1['age_interval'] = db1.age_interval.apply(lambda x: 'UKNOWN' if x == 'NC' else x)

In [ ]:
def cumulative(dataframe):
    
    cumu_num_infections = dataframe.num_infections.cumsum()
    cumu_num_hosp = dataframe.num_hosp.cumsum()
    cumu_num_uci = dataframe.num_uci.cumsum()
    cumu_num_dead = dataframe.num_dead.cumsum()

    cumulative = pd.DataFrame({'cumu_num_infections': cumu_num_infections, 'cumu_num_hosp': cumu_num_hosp, 
                               'cumu_num_uci': cumu_num_uci, 'cumu_num_dead': cumu_num_dead})
    dataframe = pd.concat([dataframe, cumulative], axis = 1)
    return dataframe

In [ ]:
def freq_rel(dataframe):
    
    dataframe['Number of infection (%)'] = [round(i/ dataframe['num_infections'].sum(),3)*100 for i in dataframe['num_infections']]
    dataframe['Number of hospitalisation (%)'] = [round(i/ dataframe['num_hosp'].sum(),3)*100 for i in dataframe['num_hosp']]
    dataframe['Number of Intensive Care Unit (%)'] = [round(i/ dataframe['num_uci'].sum(),3)*100 for i in dataframe['num_uci']]
    dataframe['Number of deaths (%)'] = [round(i/ dataframe['num_dead'].sum(),3)*100 for i in dataframe['num_dead']]

In [ ]:
def organise(column):
    
    name = db1.groupby(column).sum()
    freq_rel(name)
    
    return name

In [ ]:
def mov_7_ave(dataframe):
    dataframe['ave_7_num_infections'] = dataframe.iloc[:,1].rolling(window=7).mean()
    dataframe['ave_7_num_hosp'] = dataframe.iloc[:,2].rolling(window=7).mean()
    dataframe['ave_7_num_uci'] = dataframe.iloc[:,3].rolling(window=7).mean()
    dataframe['ave_7_num_dead'] = dataframe.iloc[:,4].rolling(window=7).mean()
    
    return dataframe

# Data Base

In [ ]:
db1 =pd.read_csv('/content/covid-19-sample.csv') 

In [ ]:
clean_data()

In [ ]:
db1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300936 entries, 0 to 300935
Data columns (total 9 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   date               300936 non-null  datetime64[ns]
 1   autonomous_region  300936 non-null  object        
 2   province           300936 non-null  object        
 3   sex                300936 non-null  object        
 4   age_interval       300936 non-null  object        
 5   num_infections     300936 non-null  int64         
 6   num_hosp           300936 non-null  int64         
 7   num_uci            300936 non-null  int64         
 8   num_dead           300936 non-null  int64         
dtypes: datetime64[ns](1), int64(4), object(4)
memory usage: 20.7+ MB


In [ ]:
bydate = db1.groupby('date').sum().reset_index()
bydate = cumulative(bydate)
bydate = mov_7_ave(bydate)

In [ ]:
bydate.head()

,date,num_infections,num_hosp,num_uci,num_dead,cumu_num_infections,cumu_num_hosp,cumu_num_uci,cumu_num_dead,ave_7_num_infections,ave_7_num_hosp,ave_7_num_uci,ave_7_num_dead
0,2020-01-18,1,1,0,0,1,1,0,0,NaN,NaN,NaN,NaN
1,2020-01-31,1,0,0,0,2,1,0,0,NaN,NaN,NaN,NaN
2,2020-02-09,1,1,0,0,3,2,0,0,NaN,NaN,NaN,NaN
3,2020-02-12,3,0,0,0,6,2,0,0,NaN,NaN,NaN,NaN
4,2020-02-21,1,0,0,0,7,2,0,0,NaN,NaN,NaN,NaN


# ARMA

In [ ]:
from statsmodels.tsa.arima_model import ARMA

In [ ]:
from sklearn.metrics import mean_squared_error as mse

# num_infections

In [ ]:
infections = pd.DataFrame(bydate.ave_7_num_infections, columns = ['ave_7_num_infections'])

In [ ]:
infections = infections.dropna().reset_index()
infections = infections.drop(columns = ['index'], axis = 1)

In [ ]:
infections

,ave_7_num_infections
0,1.428571
1,1.571429
2,1.571429
3,2.428571
4,5.285714
...,...
469,4015.142857
470,3911.142857
471,3761.857143
472,3593.714286


In [ ]:
res = []
for i in range(1, 20):
    for j in range(1,30): # max 30 -> p = 30 => erro
        for k in range(1,12):
          try:

            train, test = infections[:-i], infections[-i:]
            model=ARMA(train, order=(j, k)).fit(disp=False)
            pred=model.predict(len(train), len(infections)-1)
            res.append(((i, j, k), mse(test['ave_7_num_infections'], pred)**0.5))

          except:
            continue

/usr/local/lib/python3.7/dist-packages/statsmodels/base/model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/statsmodels/base/model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/statsmodels/base/model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/statsmodels/base/model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/statsmodels/base/model.py:492: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarni

In [1]:
res

NameError: ignored

In [ ]:
train, test = infections[:-20], infections[-20:]

In [ ]:
model=ARMA(train, order=(29, 7)).fit(disp=False)

KeyboardInterrupt: ignored

In [ ]:
model=ARMA(train, order=(28, 7)).fit(disp=False)

KeyboardInterrupt: ignored

In [ ]:
res = []
for i in range(1, 20):
    for j in range(1,30): # max 30 -> p = 30 => erro
        for k in range(1,12): # max 11
            train, test = infections[:-i], infections[-i:]
            model=ARMA(train, order=(j, k)).fit(disp=False)
            pred=model.predict(len(train), len(hosp)-1)
            res.append(((i, j, k), mse(test['ave_7_num_infections'], pred)**0.5))